In [ ]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\Panel101.dta"

df, meta = st.read_dta(path)
df.head(1)